In [ ]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00


In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
%%writefile app.py

import streamlit as st
from unsloth import FastLanguageModel
import torch

# Define a function to load the model and tokenizer
@st.cache_resource
def load_model():
    max_seq_length = 4096
    dtype = None
    load_in_4bit = True

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="devkartik/gl_llm",
        dtype=dtype,
        load_in_4bit=load_in_4bit,
    )
    FastLanguageModel.for_inference(model)  # Enable native 2x faster inference
    return model, tokenizer

# Load the model and tokenizer once
model, tokenizer = load_model()

# Alpaca prompt template
alpaca_prompt = """Bhagwad gita's knowledge is paired with an input that provides further context related to gita only. Write a response that appropriately answers and elaborate the request.

### Input:
{}

### Response:
{}"""

def extract_response(full_text):
    full_text = full_text.replace("<|end_of_text|>","")
    response_start = full_text.find("### Response:\n")
    if response_start != -1:
        return full_text[response_start + len("### Response:\n"):].strip()
    return full_text

# Initialize session state for context tracking
if 'context' not in st.session_state:
    st.session_state.context = []

# Function to format the chat history as a string
def format_context(context):
    return "\n".join([f"\nYou: {chat['user_input']}\nChatbot: {chat['response']}" for chat in context])

# Streamlit app
st.title("Bhagwad Gita Chatbot")

# Display chat history
for chat in st.session_state.context:
    st.markdown(f"**You**: {chat['user_input']}")
    st.markdown(f"**Chatbot**: {chat['response']}")

# User input
user_input = st.text_input("You:", key="user_input")

# Button to send input
if st.button("Send"):
    if user_input:
        try:
            # Prepare the input for the model
            combined_input = format_context(st.session_state.context)
            combined_input += f"\nYou: {user_input}\n"
            formatted_input = alpaca_prompt.format(combined_input, "")

            inputs = tokenizer(
                [formatted_input],
                return_tensors="pt"
            ).to("cuda")

            outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
            responses = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].strip()
            response = extract_response(responses)

            st.session_state.context.append({"user_input": user_input, "response": response})

            st.experimental_rerun()

        except torch.cuda.OutOfMemoryError:
            st.error("CUDA out of memory. Please try a shorter input or clear the context.")
        except Exception as e:
            st.error(f"An error occurred: {e}")

# Clear context button
if st.button("Clear Context"):
    st.session_state.context = []
    st.write("Context cleared.")
    st.experimental_rerun()

Overwriting app.py


In [ ]:
!npm install localtunnel
!streamlit run app.py &>/content/logs.txt &
!npx localtunnel --port 8501 & curl ipv4.icanhazip.com


up to date, audited 23 packages in 345ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
34.125.145.24
your url is: https://cool-clowns-enter.loca.lt
